In [51]:
import zipfile as zf
import matplotlib.pyplot as plt
import numpy as np
import filecmp as fcmp
import os
import scipy as sp
from PIL import Image

import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Input, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import gradient_descent_v2
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from keras.utils import np_utils
from keras.callbacks import History, EarlyStopping

from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedShuffleSplit

import cv2 as cv2

In [52]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.layers import Dense, MaxPooling2D, Conv2D, Flatten, Dropout
from keras import regularizers
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sbn
from keras.models import Sequential
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, train_test_split
import cv2
import os
from sklearn.utils import shuffle
from tqdm import tqdm
from keras import losses
from keras.wrappers.scikit_learn import KerasClassifier
import warnings
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.utils import to_categorical
import PIL.Image
import itertools
from keras_tuner import RandomSearch

In [53]:
import pandas as pd
import numpy as np

train_dataset = pd.read_csv("train.csv")
test_dataset = pd.read_csv("test.csv")


data_dir  = ""
train_zip_path = data_dir + 'Train'
test_zip_path = data_dir + 'Test'

In [54]:
dicts = np.load("dict.npy",allow_pickle='TRUE')
print (dicts)

{'0': 'Anfibio', '1': 'Artropodo', '2': 'Ave', '3': 'Mamifero', '4': 'Reptil', '5': 'Rana', '6': 'Salamandra', '7': 'Sapo', '8': 'Lombriz', '9': 'Mariposa', '10': 'Saltamontes', '11': 'Tarantula', '12': 'Viuda negra', '13': 'Buho', '14': 'Cacique', '15': 'Cisne', '16': 'Cormoran', '17': 'Ganso', '18': 'Paloma', '19': 'Pato', '20': 'Pavo', '21': 'Caballo', '22': 'Cabra', '23': 'Conejo', '24': 'Elefante', '25': 'Gato', '26': 'Leon', '27': 'Lobo', '28': 'Oso', '29': 'Perro', '30': 'Tigre', '31': 'Camaleon', '32': 'Cocodrilo', '33': 'Iguana', '34': 'Lagarto', '35': 'Serpiente', '36': 'Tortuga'}


In [55]:
shapes = []
for dirname, _, filenames in os.walk("Train"):           
    for filename in filenames:
        try:
            path = os.path.join(dirname, filename)
            image = Image.open(path)
            if image.size not in shapes:
                shapes.append(image.size)
        except:
            pass
#print(shapes)

In [56]:
import math
import numpy as np

#si es necesario
#!pip install scikit-image
import skimage.transform

# you use this just before passing any image to a CNN
# which usually expects square images
# however your input images can be of variable size
# you don't want to just squash the images to a square
# because you will lose valuable aspect ratio information
# you want to resize while preserving the aspect ratio
# these 2 functions perform this resizing behaviour
# images are assumed to be formatted as Height, Width, Channels
# we will use bound_image_dim to first bound the image to a range
# the smallest dimension will be scaled up to the min_size
# the largest dimension will be scaled down to the max_size
# then afterwards you square_pad_image to pad the image to a square
# filling the empty space with zeros

def bound_image_dim(image, min_size=None, max_size=None):
    if (max_size is not None) and \
       (min_size is not None) and \
       (max_size < min_size):
        raise ValueError('`max_size` must be >= to `min_size`')
    dtype = image.dtype
    (height, width, *_) = image.shape
    # scale the same for both height and width for fixed aspect ratio resize
    scale = 1
    # bound the smallest dimension to the min_size
    if min_size is not None:
        image_min = min(height, width)
        scale = max(1, min_size / image_min)
    # next, bound the largest dimension to the max_size
    # this must be done after bounding to the min_size
    if max_size is not None:
        image_max = max(height, width)
        if round(image_max * scale) > max_size:
            scale = max_size / image_max
    if scale != 1:
        image = skimage.transform.resize(
            image, (round(height * scale), round(width * scale)),
            order=1,
            mode='constant',
            preserve_range=True)
    return image.astype(dtype)

def square_pad_image(image, size):
    (height, width, *_) = image.shape
    if (size < height) or (size < width):
        raise ValueError('`size` must be >= to image height and image width')
    pad_height = (size - height) / 2
    pad_top = math.floor(pad_height)
    pad_bot = math.ceil(pad_height)
    pad_width = (size - width) / 2
    pad_left = math.floor(pad_width)
    pad_right = math.ceil(pad_width)
    return np.pad(
        image, ((pad_top, pad_bot), (pad_left, pad_right), (0, 0)),
        mode='constant')

In [57]:
data = []
HEIGHT = 32
WIDTH = 55
# loop over the input images
for index, imagePath in train_dataset.iterrows():
    # load the image, resize the image to be HEIGHT * WIDTH pixels (ignoring
    # aspect ratio) and store the image in the data list
    try:
        image = cv2.imread('//'+train_zip_path+'//'+imagePath[0])
        image = cv2.resize(image, (HEIGHT, WIDTH))
        print(image, imagePath[0])
        data.append(image)
    except:
        #print("failed", imagePath[0])
        continue
    print(type(image))
    
    # extract the class label from the image path and update the
    # labels list
#print(data)

In [72]:
images = list()
labels = list()

for dirname, _, filenames in os.walk(test_zip_path):           
    for filename in filenames:
        try:
            image = Image.open(os.path.join(dirname, filename))
            images.append(np.array(image, dtype=np.uint8))
            print(filename)
            print(train_dataset.loc[train_dataset['Id'] == filename.strip()].iloc[0][1])
            
            labels.append(train_dataset.loc[train_dataset['Id'] == filename].iloc[0][1])
        except:
            pass
images = np.asarray(images)
labels = np.asarray(labels)
print(labels)

image_100.jpg
image_109.jpg
image_12.jpg
image_1212.jpg
image_1220.jpg
image_1221.jpg
image_1222.jpg
image_1228.jpg
image_14.jpg
image_17.jpg
image_19.jpg
image_23.jpg
image_26.jpg
image_29.jpg
image_32.jpg
image_4305.jpg
image_4307.jpg
image_4310.jpg
image_4312.jpg
image_4337.jpg
image_4338.jpg
image_4342.jpg
image_4346.jpg
image_61.jpg
image_69.jpg
image_70.jpg
image_73.jpg
image_79.jpg
image_8.jpg
image_809.jpg
image_810.jpg
image_811.jpg
image_828.jpg
image_829.jpg
image_831.jpg
image_833.jpg
image_838.jpg
image_842.jpg
image_843.jpg
image_856.jpg
image_857.jpg
image_871.jpg
image_877.jpg
image_879.jpg
image_881.jpg
image_888.jpg
image_889.jpg
image_95.jpg
image_96.jpg
image_99.jpg
[]


c:\Users\rodri\AppData\Local\Programs\Python\Python36\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [59]:
train_dataset.loc[train_dataset['Id'] == 'image_1.jpg'].iloc[0][1]

'0 5'

In [60]:
images = images/255
images[0]

array([[[0.96470588, 0.95686275, 0.89803922],
        [0.96470588, 0.95686275, 0.89803922],
        [0.96470588, 0.96078431, 0.89019608],
        ...,
        [0.07843137, 0.07058824, 0.0745098 ],
        [0.07843137, 0.07058824, 0.0745098 ],
        [0.07843137, 0.07058824, 0.0745098 ]],

       [[0.96862745, 0.96078431, 0.90196078],
        [0.96862745, 0.96078431, 0.90196078],
        [0.96862745, 0.96470588, 0.89411765],
        ...,
        [0.0627451 , 0.05490196, 0.05882353],
        [0.0627451 , 0.05490196, 0.05882353],
        [0.0627451 , 0.05490196, 0.05882353]],

       [[0.97254902, 0.96470588, 0.90588235],
        [0.97254902, 0.96470588, 0.90588235],
        [0.97254902, 0.96862745, 0.89803922],
        ...,
        [0.0627451 , 0.05490196, 0.05882353],
        [0.05882353, 0.05098039, 0.05490196],
        [0.05882353, 0.05098039, 0.05490196]],

       ...,

       [[0.07058824, 0.07058824, 0.07058824],
        [0.08627451, 0.08627451, 0.08627451],
        [0.09411765, 0

In [61]:
from sklearn import preprocessing
labelencode = preprocessing.LabelEncoder()

labelencode.fit(labels)

LabelEncoder()

In [62]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.20, random_state=10)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((40,), (10,), (40,), (10,))

In [63]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', padding='same', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(rate=0.7))
model.add(Dense(units=3, activation='softmax'))

In [64]:
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 224, 224, 16)      448       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 112, 112, 32)      4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 54, 54, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 27, 27, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 46656)            

In [65]:
history = model.fit(x=x_train, y=to_categorical(y_train), epochs=6, validation_data=(x_test, to_categorical(y_test)), batch_size=20)

ValueError: invalid literal for int() with base 10: '0 5'